In [130]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import math
import random

In [131]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))
colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
colors[labels_index['BackG']] = 1.

In [132]:
# build training data

# sift_dir = '/oasis/projects/nsf/csd395/yuncong/Brain/learning/sift'
# sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift'
sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

train_data = []
train_labels = []
train_fnames = []

# n_sample = 1000
n_sample = 100

for name in labels[1:]:
# for name in ['7n', '5N', 'sp5', 'VLL']:
# for name in ['7n', '5N', 'sp5']:

    train_hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
    train_hists1 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l1.bp' % {'name': name})
    train_hists2 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l2.bp' % {'name': name})
    
    train_files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})
#     print (train_hists0.shape, train_hists1.shape, train_hists2.shape, train_files.shape)
    
    n_train = train_hists0.shape[0]
    print (name, n_train),

    #     train_hists = np.c_[train_hists0, train_hists1.reshape((n_train, -1)), train_hists2.reshape((n_train, -1))]

    random_indices = np.random.choice(range(n_train), min(n_train, n_sample), replace=False)
    n_train = len(random_indices)
    train_hists = np.c_[train_hists0[random_indices], 
                        train_hists1[random_indices].reshape((n_train, -1)), 
                        train_hists2[random_indices].reshape((n_train, -1))]
    train_files = train_files[random_indices].reshape((n_train,))
    
    train_data.append(train_hists)
    train_labels.append(np.ones((n_train, )) * labels_index[name])
    train_fnames.append(train_files)
    print n_train

train_data = np.concatenate(train_data)
train_labels = np.concatenate(train_labels)
train_fnames = np.concatenate(train_fnames)
n_train = train_data.shape[0]

('5N', 1440) 100
('7n', 3444) 100
('7N', 2579) 100
('12N', 1230) 100
('Pn', 3042) 100
('VLL', 1287) 100
('6N', 154) 100
('Amb', 346) 100
('R', 1082) 100
('Tz', 1387) 100
('RtTg', 2639) 100
('LRt', 1050) 100
('LC', 481) 100
('AP', 483) 100
('sp5', 3240) 100


In [133]:
print train_data.shape
print train_labels.shape
print train_fnames.shape, train_fnames.dtype
print train_fnames[0]

(1500, 4200)
(1500,)
(1500,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_5N_0299_0009.png


In [134]:
train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)
train_data_normalized = np.nan_to_num(train_data_normalized)

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':


In [135]:
# bp.pack_ndarray_file(train_data_normalized, 'preprocessed/train_data.bp')
# bp.pack_ndarray_file(train_labels, 'preprocessed/train_labels.bp')
# bp.pack_ndarray_file(train)
bp.pack_ndarray_file(train_data_normalized, sift_dir+'/preprocessed/train_data.bp')
bp.pack_ndarray_file(train_labels, sift_dir+'/preprocessed/train_labels.bp')
bp.pack_ndarray_file(train_fnames, sift_dir+'/preprocessed/train_fnames.bp')

In [136]:
# method 1
t = time.time()

def compute_intersection_kernel_oneJob(i, j1):
#     dist = .5 * np.sum(train_data_normalized + h - np.abs(train_data_normalized - h), axis=1)
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                     for i in range(n_train)))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 200s / 12k training data

train_dist_mat = np.empty((n_train, n_train))
train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
r = np.tril_indices(n_train)
train_dist_mat[r] = train_dist_mat.T[r]


compute intersection kernel: 2.523323 seconds


In [137]:
# method 2
t = time.time()

def compute_intersection_kernel_oneJob(h):
#     dist = .5 * np.sum(train_data_normalized + h - np.abs(train_data_normalized - h), axis=1)
    dist = np.minimum(train_data_normalized, h).sum(axis=1)
    return dist

# train_dist_mat = np.array(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(h) 
#                                      for h in train_data_normalized[:1000]))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 400s / 12k training data

compute intersection kernel: 0.000202 seconds


In [138]:
# method 3
t = time.time()

def compute_intersection_kernel_oneJob(i, ni, j, nj):
    dists = np.minimum(train_data_normalized[i:i+ni, None], train_data_normalized[j:j+nj]).sum(axis=-1)
    return dists

# train_dist_mat = np.empty((n_train, n_train))
# ni = 100
# nj = 100
# for j in range(0, n_train, nj):
#     train_dist_mat[:, j:j+nj] = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, ni, j, nj) 
#                                                     for i in range(0, n_train, ni)))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 400s / 12k training data

compute intersection kernel: 0.000273 seconds


In [139]:
train_dist_mat.shape

(1500, 1500)

In [140]:
# bp.pack_ndarray_file(train_dist_mat, 'preprocessed/train_spm_dist_mat.bp')
sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'
bp.pack_ndarray_file(train_dist_mat, sift_dir+'/preprocessed/train_spm_dist_mat.bp')

In [141]:
# build testing data

# MY ADDITION
sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/test'
# MY ADDITION END

stack = 'MD585'

first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

sec = first_detect_sec

# test_hists0 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l0.bp' % {'stack': stack, 'sec': sec})
# test_hists1 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l1.bp' % {'stack': stack, 'sec': sec})
# test_hists2 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l2.bp' % {'stack': stack, 'sec': sec})
test_hists0 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l0.bp' % {'stack': stack, 'sec': sec})
test_hists1 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l1.bp' % {'stack': stack, 'sec': sec})
test_hists2 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l2.bp' % {'stack': stack, 'sec': sec})

# MY ADDITION
test_fcoords = bp.unpack_ndarray_file(sift_dir + '/%(stack)s_%(sec)04d_roi1_fcoords.bp' % {'stack': stack, 'sec': sec})
# MY ADDITION END

n_test = test_hists0.shape[0]
test_hists = np.c_[test_hists0, test_hists1.reshape((n_test, -1)), test_hists2.reshape((n_test, -1))]

test_data = test_hists

In [142]:
print test_data.shape
print test_fcoords.shape

(20979, 4200)
(20979, 6)


In [143]:
test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)

In [144]:
# bp.pack_ndarray_file(test_data_normalized, 'preprocessed/%(stack)s_test_data.bp' % {'stack': stack})
sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'
bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/%(stack)s_test_data.bp' % {'stack': stack})
bp.pack_ndarray_file(test_fcoords, sift_dir+'/preprocessed/%(stack)s_test_fcoords.bp' % {'stack': stack})